Scrapping data from the word file (from the minfin.gov website)

In [1]:
import pandas as pd
from docx import Document

# function to read table from docx Minfin file
def read_table_from_docx(file_path, table_index=0):
    doc = Document(file_path) # load the word doc
    table = doc.tables[table_index] # select the table

    data = []
    keys = None
    for i, row in enumerate(table.rows):
        text = [cell.text.strip() for cell in row.cells]
        if i == 0:
            keys = text
        else:
            data.append(text)

    df = pd.DataFrame(data, columns=keys)
    return df

# Path to your Word file
file_path = 'C:/Users/lenovo/python/RNWF/data_volume/Obem_Fonda_natsionalnogo_blagosostoyaniya_-_na_01.10.2024.docx'

df = read_table_from_docx(file_path)
df

,Дата,в млрд долл. США,в млрд рублей,в процентах к ВВП**
0,01.10.2024,"137,92","12 787,13","6,7%"
1,01.09.2024,"133,42","12 165,77","6,4%"
2,01.08.2024,"142,22","12 277,51","6,4%"
3,01.07.2024,"146,96","12 601,67","6,6%"
4,01.06.2024,"141,49","12 703,81","6,6%"
...,...,...,...,...
196,01.06.2008,"32,60","773,93","1,9%"
197,01.05.2008,"32,72","773,82","1,9%"
198,01.04.2008,"32,90","773,57","1,9%"
199,01.03.2008,"32,22","777,03","1,9%"


In [2]:
df = df.rename(columns={df.columns[0]: "Date", df.columns[1]: "amount_blnUSD",df.columns[2]: 'amount_blnRUB'})
df = df.drop(columns=df.columns[3])
df['amount_blnRUB'] = df['amount_blnRUB'].str.replace(r'[***]', '', regex=True)
df['amount_blnUSD'] = df['amount_blnUSD'].str.replace(r'[***]', '', regex=True)
df['amount_blnRUB'] = df['amount_blnRUB'].str.replace(r'[ ,\xa0]+', '', regex=True)
df['amount_blnUSD'] = df['amount_blnUSD'].str.replace(',', '', regex=True).astype(float)
df['amount_blnRUB'] = df['amount_blnRUB'].astype(int) / 100
df['amount_blnUSD'] = df['amount_blnUSD'].astype(int) / 100

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
df['Date(m,Y)'] = df['Date'].dt.strftime('%B, %Y')
df

,Date,amount_blnUSD,amount_blnRUB,"Date(m,Y)"
0,2024-10-01,137.92,12787.13,"October, 2024"
1,2024-09-01,133.42,12165.77,"September, 2024"
2,2024-08-01,142.22,12277.51,"August, 2024"
3,2024-07-01,146.96,12601.67,"July, 2024"
4,2024-06-01,141.49,12703.81,"June, 2024"
...,...,...,...,...
196,2008-06-01,32.60,773.93,"June, 2008"
197,2008-05-01,32.72,773.82,"May, 2008"
198,2008-04-01,32.90,773.57,"April, 2008"
199,2008-03-01,32.22,777.03,"March, 2008"


In [4]:
df.to_csv("C:/Users/lenovo/python/RNWF/data/rnwf.csv",encoding="utf-8-sig")